In [68]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [69]:
def plot_candles(df_plot, mode=None):
    """  Create plot outputs, mode (markers/line) optional"""

    if mode == None:
        mode = "markers"

    plot_colours_buy = ['#043ef9', '#eb5334', '#34eb37']
    plot_colours_sell = ['white', 'red', 'yellow']

    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#24A06B",
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color="#2EC886",
        decreasing_line_color="#FF3A4C"
        ))

    
    fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                    margin=dict(l=10, b=10, t=30, r=10), 
                    font=dict(size=10, color="#e1e1e1"),
                    title=f"Candle {mode} Chart")
    fig.update_xaxes(gridcolor="#1f292f",
                    showgrid=True,
                    fixedrange=True,
                    rangeslider=dict(visible=False),
                    rangebreaks=[
                        dict(bounds=["sat", "mon"])
                        ]
                    )
    fig.update_yaxes(gridcolor="#1f292f",
                    showgrid=True)

    fig.show()

Note:  utils.get_histdf_fromcsv is new.  It, simply, returns DataFrame having parsed time "string" back to datetime format.  There are other - more efficient - ways we could do this...

In [70]:
pair = "USD_JPY"
granularity = "H4"
df_raw = utils.get_histdf_fromcsv(pair, granularity)
df_raw.shape

(4675, 15)

In [71]:
df_raw.columns

Index(['Unnamed: 0', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l', 'bid_c',
       'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [72]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [73]:
df = df_raw[cols].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4675 entries, 0 to 4674
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    4675 non-null   datetime64[ns, tzutc()]
 1   mid_o   4675 non-null   float64                
 2   mid_h   4675 non-null   float64                
 3   mid_l   4675 non-null   float64                
 4   mid_c   4675 non-null   float64                
dtypes: datetime64[ns, tzutc()](1), float64(4)
memory usage: 182.7 KB


In [74]:
def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_h - row.mid_o

def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l


def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] =  df.BODY_RANGE / df.RANGE
    df['DIRECTION'] =  df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x >= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis = 1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis = 1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE
    return df


In [75]:
df = apply_stats(df)

In [76]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,DIST_TOP,DIST_BOTTOM,DIST_TOP_PERC,DIST_BOTTOM_PERC
0,2020-01-01 22:00:00+00:00,108.651,108.761,108.607,108.673,0.154,0.022,108.684,0.142857,1,0.088,0.044,0.571429,0.285714
1,2020-01-02 02:00:00+00:00,108.671,108.758,108.658,108.742,0.100,0.071,108.708,0.710000,1,0.016,0.013,0.160000,0.130000
2,2020-01-02 06:00:00+00:00,108.744,108.842,108.696,108.839,0.146,0.095,108.769,0.650685,1,0.003,0.048,0.020548,0.328767
3,2020-01-02 10:00:00+00:00,108.836,108.866,108.658,108.694,0.208,0.142,108.762,0.682692,-1,0.030,0.036,0.144231,0.173077
4,2020-01-02 14:00:00+00:00,108.697,108.748,108.212,108.529,0.536,0.168,108.480,0.313433,-1,0.051,0.317,0.095149,0.591418


In [77]:
plot_candles(df)